## Content based modelling - Neural Networks data preprocessing

The following code preprocesses the data required for the neural network recommendation engine.

This notebook will query data from the SQL data warehous. This is required to extract the relevant features needed for the neural networks recommendation engine. 

In [ ]:
#First we configure the project and access to the database

#Import all required libraries
import os
import tensorflow as tf
import numpy as np
from google.cloud import bigquery 

#Environment variables 
PROJECT = 'astute-veld-253418' 
BUCKET = 'masters-research' 
REGION = 'us' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '2.1'

In [ ]:
%%bash
gcloud  config  set project $PROJECT
gcloud config set compute/region $REGION

For Tensorflow model training, we need the data in csv form to feed into the model, so we implement a helper function to assist with writing to file.

In [ ]:
def write_list_to_disk(my_list, filename):
  with open(filename, 'w') as f:
    for item in my_list:
        line = "%s\n" % item
        f.write(line)

### Import the data from the data warehouse

The original dataset is stored in a seperate warehouse and needs to be queried into the notebook environment.

In [ ]:
#Query to retrieve the occupations variables
sql="""
#standardSQL
SELECT distinct Occupation_Grouping  FROM `astute-veld-253418.Masters.pr2_Test20` 
"""

Occupation_list = bigquery.Client().query(sql).to_dataframe()['Occupation_Grouping'].tolist()
write_list_to_disk(Occupation_list, "occupation_list.txt")
print("Some sample occupations are {}".format(Occupation_list[:3]))
print("The total number of occupations are {}".format(len(Occupation_list)))

In [ ]:
#Query to retrieve the policies variable
sql="""
#standardSQL
SELECT distinct TypeName  FROM `astute-veld-253418.Masters.pr2_Test20` 
"""

Policy_list = bigquery.Client().query(sql).to_dataframe()['TypeName'].tolist()
write_list_to_disk(Policy_list, "policy_list.txt")
print("Some sample policies are {}".format(Policy_list[:3]))
print("The total number of policies are {}".format(len(Policy_list)))

In [ ]:
#Query to retrieve the gender variable
sql="""
#standardSQL
SELECT distinct Gender  FROM `astute-veld-253418.Masters.pr2_Test20` 
"""
gender_list = bigquery.Client().query(sql).to_dataframe()['Gender'].tolist()
write_list_to_disk(gender_list, "gender.txt")
print(gender_list)

In [ ]:
# Query to retrieve the Habit Variable
sql="""
#standardSQL
SELECT distinct Habit  FROM `astute-veld-253418.Masters.pr2_Test20` 
"""
habit_list = bigquery.Client().query(sql).to_dataframe()['Habit'].tolist()
write_list_to_disk(habit_list, "habit.txt")
print(habit_list)

### Create train and test sets.

Now we actually build the training and test datasets. We use a 80/20 split between this.

In [ ]:
#Retrieve training data
sql="""
SELECT * Except (Mem_ID) FROM `astute-veld-253418.Masters.pr2_Training80` 
"""
training_set_df = bigquery.Client().query(sql).to_dataframe()
training_set_df.to_csv('training_set.csv', header=False, index=False, encoding='utf-8')
training_set_df.head()

In [ ]:
#Retrieve test data
sql="""
SELECT * Except (Mem_ID) FROM `astute-veld-253418.Masters.pr2_Test20`  
"""
test_set_df = bigquery.Client().query(sql).to_dataframe()
test_set_df.to_csv('test_set.csv', header=False, index=False, encoding='utf-8')
test_set_df.head()

We now can look at the CSVs generated to ensure the data is correct

In [ ]:
%%bash
wc -l *_set.csv

In [ ]:
!head *_set.csv